In [19]:
import keras
import numpy as np
import tensorflow as tf
from keras import layers
from sklearn.metrics import classification_report
from keras import layers, models, regularizers
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau
import keras_tuner as kt
from tensorflow.keras.metrics import Precision, Recall
import tensorflow.keras.backend as K
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier


2024-07-02 10:21:41.090736: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 10:21:41.195312: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 10:21:41.196198: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-02 10:21:41.380925: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
def transcribe_audio(file_path):
    # Cargar un archivo de audio y convertirlo a un tensor
    waveform, sample_rate = torchaudio.load(file_path)

    # Resamplear el audio si es necesario
    if sample_rate != 16000:
        waveform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=16000)(waveform)

    # Tokenizar la entrada
    input_values = tokenizer(waveform.squeeze().numpy(), return_tensors="pt").input_values

    # Realizar la inferencia
    with torch.no_grad():
        logits = model(input_values).logits

    # Obtener la transcripción predicha
    predicted_ids = torch.argmax(logits, dim=-1)
    transcription = tokenizer.decode(predicted_ids[0])

    return transcription

In [3]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
import numpy as np
physical_devices = tf.config.list_physical_devices('GPU')


class AudioProcessor:
    def __init__(self, path_df, path_audio):
        self.path_df = "df_filter.csv"
        self.path_audio = path_audio
        self.load_data()

    def load_data(self):
        df = pd.read_csv(self.path_df)
        df["file_cut_path"] = self.path_audio + df["file_name"]
        df = df.reset_index(drop=True)
        self.dataset = df["file_cut_path"]

        self.log = []
        for i in range(len(df)):
            self.log.append(torch.load("torch_files14/" + f"torch_files14file{i}.pt"))

        self.y = df["label"]
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(
            self.log, self.y, random_state=42, stratify=self.y
        )

        self.process_tensors()

    def process_tensors(self):
        for i in range(len(self.X_train)):
            self.X_train[i] = self.X_train[i][0].detach().numpy()
            self.X_train[i] = np.concatenate(self.X_train[i])

        for i in range(len(self.X_test)):
            self.X_test[i] = self.X_test[i][0].detach().numpy()
            self.X_test[i] = np.concatenate(self.X_test[i])

2024-07-02 07:33:05.845385: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 07:33:05.850581: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-07-02 07:33:05.850624: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.


In [4]:
processor = AudioProcessor(path_df="./df_filter.csv", path_audio="../audios/")
X_train   = processor.X_train
y_train  = processor.y_train
X_test = processor.X_test
y_test = processor.y_test

X_train_formated = np.array(X_train)
y_train_formated = np.array(y_train)
X_test_formated = np.array(X_test)
y_test_formated = np.array(y_test)

print("X_train_formated")
X_train_formated.shape
print("y_train_formated")
y_train_formated.shape
print("X_test_formated")
X_test_formated.shape
print("y_test_formated")
y_test_formated.shape


print("Datos cargados")

X_train_formated
y_train_formated
X_test_formated
y_test_formated
Datos cargados


In [5]:
y_train_formated.shape

(1794,)

In [ ]:

# Definir parámetros adicionales
params = {
    'eval_metric': 'error',
    'eta': 0.1,
    'max_depth': 3,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'tree_method': 'gpu_hist'
}

# Inicialización y entrenamiento del clasificador XGBoost
classifier = XGBClassifier(objective="binary:logistic", seed=42, **params)
classifier.fit(X_train_formated, y_train_formated, verbose=3, eval_set=[(X_test_formated, y_test_formated)])

# Predecir y evaluar el modelo
y_pred = classifier.predict(X_test_formated)
score = accuracy_score(y_test_trans, y_pred)
print(f"Accuracy: {score}")

# Mostrar otras métricas si están definidas en 'metricas'
for metric in metricas:
    print(metric)